<a href="https://colab.research.google.com/github/Chiamakac/TRAININGS/blob/main/IgboBERT%202.0/Training_IboBERT_2_0_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Training an Igbo language model from scratch using Transformers and Tokenizers**

**1. Getting the data.**

In [ ]:
!wget -c https://github.com/IgnatiusEzeani/IGBONLP/raw/master/ig_monoling/text.zip
!wget -c https://raw.githubusercontent.com/chiamaka249/lacuna_pos_ner/main/language_corpus/ibo/ibo.txt
!wget -c https://raw.githubusercontent.com/Chiamakac/IboBETA/main/config.json?token=GHSAT0AAAAAAB5DFIJTG6K26ACHVLWSCFIAY6JXY4Q
!wget -c https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/Projection_Work/TAG_ENG/Igbo%20Sents_all.txt

--2023-01-19 22:04:47--  https://github.com/IgnatiusEzeani/IGBONLP/raw/master/ig_monoling/text.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IgnatiusEzeani/IGBONLP/master/ig_monoling/text.zip [following]
--2023-01-19 22:04:47--  https://raw.githubusercontent.com/IgnatiusEzeani/IGBONLP/master/ig_monoling/text.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7604282 (7.3M) [application/zip]
Saving to: ‘text.zip’

text.zip            100%[===================>]   7.25M  --.-KB/s    in 0.04s   

2023-01-19 22:04:48 (173 MB/s) - ‘text.zip’ saved [7604282/7604282]

--2023-01-19 22:04:48--  htt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Unzip the zipped file and remove the zipped file after unzipping
import zipfile
import os


def unzip(zipfilename):
  try:
    with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
      zip_ref.extractall(zipfilename[:-4])
      return f"'{zipfilename}' unzipped!"
  except FileNotFoundError:
    print(f"Cannot find '{zipfilename}' file")

unzip("text.zip")
!rm text.zip

In [ ]:
#copies the file "ibo.txt" into the folder "text"
import shutil
shutil.move('/content/ibo.txt', '/content/text')



'/content/text/ibo.txt'

In [ ]:
import shutil
shutil.move('/content/Igbo Sents_all.txt', '/content/text')

'/content/text/Igbo Sents_all.txt'

In [ ]:

# import os
#import shutil
dir_name = "/content/text"
text=""
for fname in os.listdir(dir_name):
  fname = os.path.join(dir_name, fname)
  with open(fname, "r", encoding="utf8") as datafile:
    text = text+"\n"+datafile.read()

with open("data.txt", "w", encoding="utf8") as datafile:
  datafile.write(text)

shutil.rmtree("text")

**2.  Import Transformers, Tokenizer and Train the tokenizer**

In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow

# Install `transformers` from master stating the version
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
  Successfully uninstalled tensorflow-2.9.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-v_5j3rey
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-v_5j3rey
  Resolved https://github.com/huggingface/transformers to commit 862888a35834527fed61beaf42373423ffdbd216
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.4 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=6271962 sha256=9264fd9319d8ec2dbd246a7be2c49f402718f25da86779f4401d9ad1e7a93

In [ ]:

%%time 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

# Describing the path to all of our Igbo data 
paths = [str(x) for x in Path(".").glob("**/*.txt")]
print(paths)

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",#Beginning of sequence (BOS) or classifier (CLS) token
    "<pad>",# Padding token
    "</s>",#End of sequence (EOS) or seperator (SEP) token
    "<unk>",# Unknown token
    "<mask>", # Masking token
])

['data.txt', 'drive/MyDrive/TEXT.txt', 'drive/MyDrive/IBO_BETA/LREC FINAL TRAINING/bert-base-multilingual-cased checkpoint-5600/vocab.txt', 'drive/MyDrive/IBO_BETA/LREC FINAL TRAINING/distilbert-base-uncased checkpoint-5600/vocab.txt', 'drive/MyDrive/IBO_BETA/LREC FINAL TRAINING/IgboBert-finetuned-ner checkpoint-5600/merges.txt', 'drive/MyDrive/IBO_BETA/LREC FINAL TRAINING/IgboBert_1e-4-finetuned-ner checkpoint-5600/merges.txt', 'drive/MyDrive/IBO_BETA/Results for IgboBert/results/checkpoint-500/merges.txt', 'drive/MyDrive/IBO_BETA/IgboBert/merges.txt', 'drive/MyDrive/IboBERT_2.0/IgboBert_2.0/merges.txt']
CPU times: user 28.2 s, sys: 9.15 s, total: 37.4 s
Wall time: 16.4 s


In [ ]:
#Our tokenizer is now ready and we have two files that define our new IgboBert tokenizer( a vocab.json-which is a list of the most frequent tokens ranked by frequency and a merges.txt list of merges)
#we then save the file for later use

!mkdir IgboBert_2.0
tokenizer.save_model("IgboBert_2.0")

['IgboBert_2.0/vocab.json', 'IgboBert_2.0/merges.txt']

In [ ]:
shutil.move('/content/config.json', '/content/IgboBert_2.0')

'/content/IgboBert_2.0/config.json'

**2. Initializing the Tokenizer**

Let's initialize our tokenizer. This way we can use it as we would use any other from_pretrained tokenizer.

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./IgboBert_2.0/vocab.json",
    "./IgboBert_2.0/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.encode("Simone gara ụka ụnyahụ guọ egwu ma ga-kwa taa.", "Aha ya bụ ifeoma.").tokens

In [ ]:
# Check that we have a GPU
!nvidia-smi

Thu Jan 19 22:11:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [ ]:
#For training, we need a raw (not pre-trained) BERTLMHeadModel. 
#To create that, we first need to create a RoBERTa config object to describe the parameters we’d like to initialize IgboBERT with.

from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)


In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./IgboBert_2.0", max_len=512, config=config)

In [ ]:
#We import and initialize our RoBERTa model with a language modeling (LM) head.

from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()
# => 83 million parameters

83504416

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = "/content/data.txt",
    block_size = 128
)

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: user 31.8 s, sys: 1.25 s, total: 33 s
Wall time: 15.9 s


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./IgboBert_2.0",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 402581
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 31455
  Number of trainable parameters = 83504416
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future versio

Step,Training Loss
500,6.448100
1000,5.348400
1500,4.907400
2000,4.616600
2500,4.390200
3000,4.232300
3500,4.071000
4000,3.943100
4500,3.865500
5000,3.791300


Saving model checkpoint to ./IgboBert_2.0/checkpoint-10000
Configuration saved in ./IgboBert_2.0/checkpoint-10000/config.json
Model weights saved in ./IgboBert_2.0/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to ./IgboBert_2.0/checkpoint-20000
Configuration saved in ./IgboBert_2.0/checkpoint-20000/config.json
Model weights saved in ./IgboBert_2.0/checkpoint-20000/pytorch_model.bin
Saving model checkpoint to ./IgboBert_2.0/checkpoint-30000
Configuration saved in ./IgboBert_2.0/checkpoint-30000/config.json
Model weights saved in ./IgboBert_2.0/checkpoint-30000/pytorch_model.bin
Deleting older checkpoint [IgboBert_2.0/checkpoint-10000] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 5h 37min 21s, sys: 30.3 s, total: 5h 37min 52s
Wall time: 5h 36min 37s


TrainOutput(global_step=31455, training_loss=3.216083864846606, metrics={'train_runtime': 20197.041, 'train_samples_per_second': 99.663, 'train_steps_per_second': 1.557, 'total_flos': 3.5219237384375616e+16, 'train_loss': 3.216083864846606, 'epoch': 5.0})

In [ ]:
trainer.save_model("./IgboBert_2.0")

Saving model checkpoint to ./IgboBert_2.0
Configuration saved in ./IgboBert_2.0/config.json
Model weights saved in ./IgboBert_2.0/pytorch_model.bin


# **4. Test the Model**

We first initialize a pipeline object, using the 'fill-mask' argument. Then begin testing our model like so



In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./IgboBert_2.0",
    tokenizer="./IgboBert_2.0"
)

loading configuration file ./IgboBert_2.0/config.json
Model config RobertaConfig {
  "_name_or_path": "./IgboBert_2.0",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file ./IgboBert_2.0/config.json
Model config RobertaConfig {
  "_name_or_path": "./IgboBert_2.0",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classi

In [ ]:
# The sun <mask>.
# =>

fill_mask("Abụ m Maazị <mask>.") #= okafor/Ọkafọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.02693532407283783,
  'token': 355,
  'token_str': ' A',
  'sequence': 'Abụ m Maazị A.'},
 {'score': 0.011763395741581917,
  'token': 455,
  'token_str': ' M',
  'sequence': 'Abụ m Maazị M.'},
 {'score': 0.009665265679359436,
  'token': 610,
  'token_str': ' T',
  'sequence': 'Abụ m Maazị T.'},
 {'score': 0.008098028600215912,
  'token': 3380,
  'token_str': ' Nwaanyị',
  'sequence': 'Abụ m Maazị Nwaanyị.'},
 {'score': 0.007046069949865341,
  'token': 6993,
  'token_str': ' Ọkafọ',
  'sequence': 'Abụ m Maazị Ọkafọ.'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na <mask> ji na akara.") #= eri
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.18506872653961182,
  'token': 292,
  'token_str': ' ya',
  'sequence': 'Nwaanyị na ya ji na akara.'},
 {'score': 0.037972841411828995,
  'token': 300,
  'token_str': ' ha',
  'sequence': 'Nwaanyị na ha ji na akara.'},
 {'score': 0.0356743261218071,
  'token': 911,
  'token_str': ' Naịjirịa',
  'sequence': 'Nwaanyị na Naịjirịa ji na akara.'},
 {'score': 0.0318823978304863,
  'token': 671,
  'token_str': ' nwaanyị',
  'sequence': 'Nwaanyị na nwaanyị ji na akara.'},
 {'score': 0.021502790972590446,
  'token': 774,
  'token_str': ' nwunye',
  'sequence': 'Nwaanyị na nwunye ji na akara.'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Chineke ga- ebibikwa ndị niile na- eme ihe <mask>.") #=ọjọọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.1908637136220932,
  'token': 760,
  'token_str': ' ọjọọ',
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọjọọ.'},
 {'score': 0.144007608294487,
  'token': 266,
  'token_str': ' a',
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe a.'},
 {'score': 0.13014595210552216,
  'token': 445,
  'token_str': ' niile',
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe niile.'},
 {'score': 0.10273794829845428,
  'token': 518,
  'token_str': ' ọma',
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọma.'},
 {'score': 0.02662363275885582,
  'token': 412,
  'token_str': ' anya',
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe anya.'}]

In [ ]:
fill_mask("ọba akwụkwọ Ọkammụta Kenneth Dike dị <mask>.") #n'Awka

# This is the beginning of a beautiful <mask>.
# =>

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na eri <mask> na akara.") #= ji
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

In [ ]:
# The sun <mask>.
# =>

fill_mask("Gaanụ mee ndị <mask> niile ka ha bụrụ ndị na- eso ụzọ m  .") #= mba


In [ ]:
# The sun <mask>.
# =>

fill_mask("Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ <mask>.") #= Izrel


In [ ]:
# The sun <mask>.
# =>

fill_mask("Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka <mask> Haram.") #= Boko


In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado <mask> ọhụrụ a.") #= iwu


In [ ]:
# The sun <mask>.
# =>

fill_mask(" <mask> sị ka ehiwe ụlọikpe pụrụiche maka mpụ.") #= Buhari


In [ ]:
# The sun <mask>.
# =>

fill_mask("Ala <mask>  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.") #= Naịjirịa


In [ ]:
# The sun <mask>.
# =>

fill_mask("Ọ bụ <mask>a ka a na- arịa .") #= mmadụ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

In [ ]:
#mount gdrive
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
#move model to gdrive
shutil.move('/content/IgboBert_2.0','/content/drive/MyDrive/IboBERT_2.0')

'/content/drive/MyDrive/IboBERT_2.0/IgboBert_2.0'